In [ ]:
# Basic Entropy based Feature Selection

In [2]:
import pandas as pd
import numpy as np
import csv
import gc
import os
import copy
#from sklearn.model_selection import train_test_split
from random import randint
from bisect import bisect
from math import log

In [3]:
def partition(array, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i][1] >= array[begin][1]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    return pivot



def quicksort(array, begin=0, end=None):
    if end is None:
        end = len(array) - 1
    def _quicksort(array, begin, end):
        if begin >= end:
            return
        pivot = partition(array, begin, end)
        _quicksort(array, begin, pivot-1)
        _quicksort(array, pivot+1, end)
    return _quicksort(array, begin, end)

In [12]:
def entropy(freq1,freq2):
    # print(freq1)
    # print(freq2)
    div1 = freq1/(freq1+freq2)
    div2 = freq2/(freq1+freq2)
    if (div1==0 and div2==0):
        return 0
    else: 
        if (div1==0):
            return -(div2*log(div2,2))
        else:
            if (div2==0):
                return -(div1*log(div1,2))
    
    result = -(div1*log(div1,2))-(div2*log(div2,2))
    return result

In [10]:
def probability(freq,column):
    # freq = sum(map(lambda x: 1 if x==value else 0,column))
    return freq/column.size

In [6]:
def cross_freq(column,target_class,value):
    pos_cant=0
    neg_cant=0
    for i in range(0,column.size):
        if column[i]==value:
            if target_class[i]==1:
                pos_cant+=1
            else:
                neg_cant+=1
    return [pos_cant,neg_cant]

In [7]:
def entropy_vs(column,target_class):
    # we know the other columns only have 0,1 and -1  values
    cross_freq_0 = cross_freq(column,target_class,0)
    freq_0=cross_freq_0[0]+cross_freq_0[1]
    member_0 = probability(freq_0,column)*entropy(cross_freq_0[0],cross_freq_0[1])
    
    
    cross_freq_1 = cross_freq(column,target_class,1)
    freq_1=cross_freq_1[0]+cross_freq_1[1]
    member_1 = probability(freq_1,column)*entropy(cross_freq_1[0],cross_freq_1[1])
    
    cross_freq_minus_1 = cross_freq(column,target_class,-1)
    freq_minus_1=cross_freq_minus_1[0]+cross_freq_minus_1[1]
    member_minus_1 = probability(freq_minus_1,column)*entropy(cross_freq_minus_1[0],cross_freq_minus_1[1])
    
    return member_0 + member_1 + member_minus_1

In [6]:
directory='/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches/censored_CTNNB1_Target'
if not os.path.exists(directory):
    print("Bad routing.")

In [7]:
def takeSecond(elem):
    return elem[1]

In [ ]:

main_directory= '/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches'
main_save_folder="/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/FeatureSelection/Entropy"
main_data={}
for folder in os.listdir(main_directory):
    directory=main_directory+'/'+folder
    save_folder=main_save_folder+'/'+folder
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    main_data[folder]=[]
    for filename in os.listdir(directory):
        print("Have entered in "+filename)
        data=pd.read_csv(directory+'/'+filename)
        data['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1),data['Pvalue']))
        del data['Unnamed: 0']
        del data['TrueIndex']
        del data['CauseGene']
        del data['EffectGene']
        del data['Replicate']
        del data['Treatment']
        preliminary_results=[]
        for key in data:
            if key != 'Pvalue':
                preliminary_results.append([key,entropy_vs(data[key],data['Pvalue'])])
        # Should first sort the results
        quicksort(preliminary_results)
        main_data[folder].append(preliminary_results)
        # write out
        df = pd.DataFrame(preliminary_results)
        df.to_csv(save_folder+"/results_"+filename)
        del df
        del data
        gc.collect()

# process main_data
# top 100
for key in main_data:
    for data in main_data[key]:
        data=data[:100]

Have entered in batch_41.csv
Have entered in batch_9.csv
Have entered in batch_23.csv
Have entered in batch_26.csv
Have entered in batch_71.csv
Have entered in batch_99.csv
Have entered in batch_54.csv
Have entered in batch_10.csv
Have entered in batch_39.csv
Have entered in batch_11.csv
Have entered in batch_1.csv
Have entered in batch_53.csv
Have entered in batch_78.csv
Have entered in batch_51.csv
Have entered in batch_102.csv
Have entered in batch_47.csv
Have entered in batch_13.csv
Have entered in batch_85.csv
Have entered in batch_91.csv
Have entered in batch_57.csv
Have entered in batch_12.csv
Have entered in batch_7.csv
Have entered in batch_6.csv
Have entered in batch_44.csv
Have entered in batch_52.csv
Have entered in batch_67.csv
Have entered in batch_101.csv
Have entered in batch_79.csv
Have entered in batch_76.csv
Have entered in batch_103.csv
Have entered in batch_40.csv
